# CSV a txts

Vamos a generar los inserts para las tablas de Autosummit Perú SAC

Importar librerías

In [1]:
import pandas as pd
import numpy as np

Abrir CSV

In [2]:
ds_ASP = pd.read_csv('ASP.csv', sep=',')

Ejemplos

In [3]:
ds_ASP['Cliente']

0                (42206340) DANIELLA MARIA BOLAÑOS GAMERO
1                         (20100115663) PANDERO S.A. EAFC
2       (09468059) KATIA NATHALI DE LOAYZA WONG DE PAC...
3        (46472213) JHONATHAN MITCHELL ANTEZANA ESCALANTE
4                (42607724) KRISCIA ZULAY REATEGUI ZAMORA
                              ...                        
1253                                                  NaN
1254                                                  NaN
1255                                                  NaN
1256                                                  NaN
1257                                                  NaN
Name: Cliente, Length: 1258, dtype: object

<h3>Filtrado de colores</h3>

In [4]:
colors_rep = ds_ASP['Color']
colors_diccionario = {}  # color limpio -> id_color
id_color = 1

for c in colors_rep:
    if isinstance(c, str) and c.strip() != '' and c.lower() != 'nan':
        color = c.strip().upper()
        if color not in colors_diccionario:
            colors_diccionario[color] = id_color
            id_color += 1

text_colors = '-- migrate:up\n\n'
for color, id_color in colors_diccionario.items():
    c_escaped = color.replace("'", "''")
    text_colors += f"INSERT INTO color (id_color, nombre) VALUES ({id_color}, '{c_escaped}');\n"

text_colors += '\n-- migrate:down\nDELETE FROM color;'

with open('inserts_colors.sql', 'w', encoding='utf-8') as f:
    f.write(text_colors)


<p>Filtrado por Asesor </p>

In [5]:
import re
asesores_rep = ds_ASP['Asesor']
asesores = []

#En caso tenga nombres "raros" el asesor
PALABRAS_INVALIDAS = {
    "PDI", "EXHIBICION", "ATE", "CASO", "ENTREGA"
}

#Almacenar todo en diccionario - para evitar duplicados
asesores = {} 

def es_asesor_valido(texto):
    if not isinstance(texto, str):
        return False

    texto = texto.strip().upper()

    # Debe tener al menos un espacio (nombre + apellido)
    if " " not in texto:
        return False

    # No debe contener comas ni números
    if "," in texto or re.search(r"\d", texto):
        return False

    # No debe contener palabras inválidas
    for palabra in PALABRAS_INVALIDAS:
        if palabra in texto:
            return False

    # Solo letras y espacios
    if not re.match(r"^[A-ZÁÉÍÓÚÑ ]+$", texto):
        return False

    return True


for c in asesores_rep:
    if not es_asesor_valido(c):
        continue

    c = c.strip().upper()
    nombres, apellidos = c.split(" ", 1)

    key = (nombres, apellidos)
    asesores[key] = True

sql_asesores = "-- migrate:up\n\n"
id_asesor = 1
#para recepciones
asesores_diccionario={}
for nombres, apellidos in sorted(asesores.keys()):
    nombre_completo = f"{nombres} {apellidos}"
    sql_asesores += (
        "INSERT INTO asesores (id_asesor, nombres, apellidos) "
        f"VALUES ({id_asesor}, '{nombres}', '{apellidos}');\n"
    )
    asesores_diccionario[(nombre_completo)] = id_asesor
    id_asesor += 1

sql_asesores += "\n-- migrate:down\nDELETE FROM asesores;\n"

with open("inserts_asesores.sql", "w", encoding="utf-8") as f:
    f.write(sql_asesores)


<h3>Descomposición en marca (nombre) y modelo (nombre, version, año)</h3>

<p> Considerar que Marca no está de forma explícita en el CSV. Solución: inferir y filtrar con diccionarios </p>

In [6]:
import re

# =========================
# 1. COLUMNA CSV
# =========================
modelos_rep = ds_ASP['Modelo']

# Diccionarios con IDs
marcas_dict = {}      # marca -> id_marca
modelos_dict = {}     # (modelo_base, version, traccion, anio, motor, marca) -> id_modelo
id_modelo = 1

# =========================
# CATÁLOGOS CONTROLADOS
# =========================
MAPA_MARCAS = {
    "FORD": [
        "TERRITORY", "RANGER", "F-150", "MAVERICK",
        "EXPLORER", "ESCAPE", "BRONCO",
        "MUSTANG", "EXPEDITION"
    ],
    "CHERY": [
        "TIGGO", "ARRIZO", "M7", "HIMLA"
    ]
}

VERSIONES_VALIDAS = {
    "TITANIUM", "TREND", "XLS", "XLT", "XL", "LTD",
    "PLATINUM", "RAPTOR", "BADLANDS", "LARIAT",
    "ACTIVE", "ST", "PREMIUM", "PRO", "MAX",
    "BIG", "BEND", "TREMOR", "GT"
}

TRACCIONES = {"4X2", "4X4", "AWD", "4WD"}
DESCARTES = {"MT", "AT", "CVT", "DCT", "FHEV", "MHEV", "PHEV", "GLP", "GNV"}

# =========================
# FUNCIONES
# =========================

def detectar_marca(texto: str):
    texto = texto.upper()
    for marca, modelos in MAPA_MARCAS.items():
        for m in modelos:
            if m in texto:
                return marca
    return None

def extraer_anio(texto: str):
    match = re.search(r"(20\d{2})", texto)
    return int(match.group(1)) if match else None

def separar_modelo(texto: str):
    tokens = texto.upper().split()
    motor = None
    traccion = None
    usados = set()

    for t in tokens:
        # Motor (1.5L, 2.0T, etc.)
        if re.match(r"\d\.\d(T|L)?", t):
            motor = float(re.findall(r"\d\.\d", t)[0])
            usados.add(t)

        # Tracción
        elif t in TRACCIONES:
            traccion = t
            usados.add(t)

        # Tokens técnicos descartables
        elif t in DESCARTES:
            usados.add(t)

    # limpiar tokens ya usados
    limpio = [t for t in tokens if t not in usados]
    modelo_base = limpio[0] if limpio else None
    version_tokens = [t for t in limpio[1:] if t in VERSIONES_VALIDAS]
    version_modelo = " ".join(version_tokens) if version_tokens else None

    return modelo_base, version_modelo, traccion, motor

# =========================
# PROCESAMIENTO DEL CSV
# =========================
for fila in modelos_rep:
    if not isinstance(fila, str) or fila.strip() == "":
        continue

    fila = fila.strip().upper()
    marca = detectar_marca(fila)
    anio = extraer_anio(fila)

    if not marca or not anio:
        continue

    modelo_base, version, traccion, motor = separar_modelo(fila)
    if not all([modelo_base, version, traccion, motor]):
        continue

    # Registrar marca con ID
    if marca not in marcas_dict:
        marcas_dict[marca] = len(marcas_dict) + 1
    marca_id = marcas_dict[marca]

    # Registrar modelo con ID
    key = (modelo_base, version, traccion, anio, motor, marca)
    if key not in modelos_dict:
        modelos_dict[key] = id_modelo
        id_modelo += 1

# =========================
# SQL: MARCAS
# =========================
sql_marcas = "-- migrate:up\n\n"
for marca, id_marca in marcas_dict.items():
    c_escaped = marca.replace("'", "''")
    sql_marcas += f"INSERT INTO marcas (id_marca, nombre) VALUES ({id_marca}, '{c_escaped}');\n"

sql_marcas += "\n-- migrate:down\nDELETE FROM marcas;\n"

with open("inserts_marcas.sql", "w", encoding="utf-8") as f:
    f.write(sql_marcas)

# =========================
# SQL: MODELOS
# =========================
sql_modelos = "-- migrate:up\n\n"
for (modelo_base, version, traccion, anio, motor, marca), id_modelo in modelos_dict.items():
    marca_id = marcas_dict[marca]
    modelo_base_esc = modelo_base.replace("'", "''")
    version_esc = version.replace("'", "''") if version else ''
    traccion_esc = traccion.replace("'", "''") if traccion else ''

    sql_modelos += (
        "INSERT INTO modelos "
        "(id_modelo, modelo_base, version_modelo, traccion, anio, motor, marca_id) "
        f"VALUES ({id_modelo}, '{modelo_base_esc}', '{version_esc}', "
        f"'{traccion_esc}', {anio}, {motor}, {marca_id});\n"
    )

sql_modelos += "\n-- migrate:down\nDELETE FROM modelos;\n"

with open("inserts_modelos.sql", "w", encoding="utf-8") as f:
    f.write(sql_modelos)

<h2>Filtrado de GPS</h2>
<p>Eliminar si tiene la palabra "GPS" para dejar el resto del texto </p>

In [7]:
import pandas as pd
import re

def limpiar_gps(texto):
    if not isinstance(texto, str):
        return None

    texto = texto.strip().upper()

    # Eliminar fechas entre paréntesis
    texto = re.sub(r"\(.*?\)", "", texto)

    #Eliminar comentarios
    texto = re.sub(r"\d+/\d+", "", texto)

    #PALABRAS NO ACEPTADAS: Eliminar si existen
    FiltroNoAceptado = [
        "OK", "INSTALADO", "COORDINAR", "CLIENTE",
        "NO APLICA", "CAMPAÑA", "AUTOPLAN"
    ]

    for palabra in FiltroNoAceptado:
        texto = texto.replace(palabra, "")

    #Solo queremos letras y espacios
    texto = re.sub(r"[^A-Z ]", "", texto)
    texto = re.sub(r"\s+", " ", texto).strip()

    return texto #La versión limpia del texto que pasa

def detectar_gps_base(texto):
    GPS_VALIDOS = {"COMSATEL","HUNTER", "SUPRA","PANDERO", "PROTEMAX", "MAQUISISTEMAS","EURORENTING"}
    
    # Forma limpia: 'GPS {PALABRACLAVE}'
    for palabraGPS in GPS_VALIDOS:
        if palabraGPS in texto: 
            return f"GPS {palabraGPS}"
    return None #Si no encaja, no es válido

ds = pd.read_csv("ASP.csv", encoding="utf-8")

gps_dict = {}
id_gps = 1

for g in ds['GPS']:
    nombreGPSLimpio = limpiar_gps(g)
    if not nombreGPSLimpio:
        continue

    gps_base = detectar_gps_base(nombreGPSLimpio)

    if not gps_base:
        continue 

    if gps_base not in gps_dict:
        gps_dict[gps_base] = id_gps
        id_gps += 1

# Generar SQL GPS

sql_gps = "-- migrate:up\n\n"

for nombre, id_ in gps_dict.items():
    sql_gps += (
        "INSERT INTO gps (id_gps, nombre) "
        f"VALUES ({id_}, '{nombre}');\n"
    )

sql_gps += "\n-- migrate:down\nDELETE FROM gps;\n"

with open("inserts_gps.sql", "w", encoding="utf-8") as f:
    f.write(sql_gps)


<h2>Filtrado de Clientes </h2>
<p>Al estar en cliente el nombre de empresa / persona natural + DNI/RUC El objetivos principal será separar número de identificación de identidad y el nombre, así como clasificar en caso sea persona natural o empresa </p>

In [8]:
import re

clientes_rep = ds_ASP['Cliente']

# ===================== FUNCIONES =====================

def limpiar_cliente(texto):
    """
    (20100115663) PANDERO S.A. EAFC
    """
    if not isinstance(texto, str):
        return None, None

    texto = texto.strip()

    match = re.match(r"\((\d+)\)\s*(.+)", texto)
    if not match:
        return None, None

    numero = match.group(1)
    nombre = match.group(2).strip().upper()

    return numero, nombre


PALABRAS_EMPRESA = {
    "S.A", "S.A.", "S.A.C", "SAC", "SOCIEDAD",
    "EMPRESA", "E.A.F.C", "EAFC", "CORPORACION",
    "GRUPO", "GROUP", "E.I.R.L", "SERVICIOS",
    "SRL", "S.R.L"
}

def es_empresa(nombre):
    return any(p in nombre for p in PALABRAS_EMPRESA)


# ===================== ESTRUCTURAS =====================

clientes_dict = {}          # id_cliente -> (numero, nombre)
personas = set()       # ids
empresas = set()       # ids

id_cliente = 1
id_persona = 1
id_empresa = 1

# ===================== PROCESAMIENTO =====================

for fila in clientes_rep:
    numero, nombre = limpiar_cliente(fila)

    if not numero or not nombre:
        continue

    clientes_dict[id_cliente] = (numero, nombre)

    if es_empresa(nombre):
        empresas.add(id_cliente)
    else:
        personas.add(id_cliente)

    id_cliente += 1


# ===================== SQL CLIENTES =====================

sql_clientes = "-- migrate:up\n\n"

for id_cliente, (numero, nombre) in clientes_dict.items():
    nombre = nombre.replace("'", "''")
    sql_clientes += (
        "INSERT INTO clientes (id_cliente, Numero_Identificacion, nombre) "
        f"VALUES ({id_cliente}, '{numero}', '{nombre}');\n"
    )

sql_clientes += "\n-- migrate:down\nDELETE FROM clientes;\n"

with open("inserts_clientes.sql", "w", encoding="utf-8") as f:
    f.write(sql_clientes)


# ===================== SQL PERSONA NATURAL =====================

sql_personas = "-- migrate:up\n\n"

for id_cliente in personas:
    sql_personas += (
        "INSERT INTO persona_natural (id_persona, cliente_id) "
        f"VALUES ({id_persona}, {id_cliente});\n"
    )
    id_persona += 1

sql_personas += "\n-- migrate:down\nDELETE FROM persona_natural;\n"

with open("inserts_persona_natural.sql", "w", encoding="utf-8") as f:
    f.write(sql_personas)


# ===================== SQL EMPRESA =====================

sql_empresas = "-- migrate:up\n\n"

for id_cliente in empresas:
    sql_empresas += (
        "INSERT INTO empresa (id_empresa, cliente_id) "
        f"VALUES ({id_empresa}, {id_cliente});\n"
    )
    id_empresa += 1

sql_empresas += "\n-- migrate:down\nDELETE FROM empresa;\n"

with open("inserts_empresa.sql", "w", encoding="utf-8") as f:
    f.write(sql_empresas)


<h2>Filtrado vehiculo</h2>

<p>Dado que vehículo depende de otros: colores, MODELO. GPS, etc. Y esos respectivos datos ya ha ¿n sido filtrados, se reutiliza el coidgo e inserta en diccionarios.</p>

In [25]:
import numpy as np
import pandas as pd
import re

vehiculos = ds_ASP[['PLACA', 'VIN', 'Color', 'Modelo', 'Cliente', 'GPS']].to_numpy()

text = '-- migrate:up\n\n'
id_vehiculo = 1
vehiculos_diccionario = {}

def sql_value(v):
    if v is None:
        return "NULL"
    return f"'{v}'"

for n in vehiculos:

    placa = str(n[0]).strip().upper() if pd.notna(n[0]) else None
    vin = str(n[1]).strip().upper() if pd.notna(n[1]) else None

    color_str = str(n[2]).strip().upper() if pd.notna(n[2]) else None
    colors_v = colors_diccionario.get(color_str) if color_str else None

    gps_v = None
    if pd.notna(n[5]):
        gps_str = limpiar_gps(str(n[5]))
        gps_base = detectar_gps_base(gps_str)
        gps_v = gps_dict.get(gps_base)

    modelo_v = None
    if pd.notna(n[3]):
        modelo_str = str(n[3]).strip().upper()
        for (modelo, version, traccion, anio, motor, marca), mid in modelos_dict.items():
            if modelo.upper() in modelo_str:
                modelo_v = mid
                break

    cliente_v = None
    if pd.notna(n[4]):
        cliente_str = str(n[4]).strip().upper()
        match = re.search(r'\((\d+)\)', cliente_str)
        if match:
            doc = match.group(1)
            for cid, (dni, nombre) in clientes_dict.items():
                if dni == doc:
                    cliente_v = cid
                    break

    if cliente_v is None:
        continue

    valores = [placa, vin, colors_v, modelo_v, gps_v, cliente_v]
    if all(v is None for v in valores):
        continue

    placa_sql = sql_value(placa)
    vin_sql = sql_value(vin)
    color_sql = colors_v if colors_v is not None else "NULL"
    modelo_sql = modelo_v if modelo_v is not None else "NULL"
    gps_sql = gps_v if gps_v is not None else "NULL"
    cliente_sql = cliente_v if cliente_v is not None else "NULL"

    text += (
        f"INSERT INTO vehiculos "
        f"(id_vehiculo, placa, vin, color_id, modelo_id, gps_id, cliente_id) "
        f"VALUES ({id_vehiculo}, {placa_sql}, {vin_sql}, "
        f"{color_sql}, {modelo_sql}, {gps_sql}, {cliente_sql});\n"
    )

    vehiculos_diccionario[id_vehiculo] = (
        placa,
        vin,
        colors_v,
        modelo_v,
        gps_v,
        cliente_v
    )

    id_vehiculo += 1

text += '\n-- migrate:down\nDELETE FROM vehiculos;\n'

with open('inserts_vehiculos.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text)

<h2>Distritos y Ubicaciones</h2>

<p>Dada la estructura de los datos, se ha optado por crear un diccionario que asocie cada ubicacion con un distrito, en caso no especifique, se optará por NULL.</p>
<p>Primero identificar distritos y luego de ahí obtener ubicaciones</p>

In [10]:
import re
import random
import pandas as pd


ubicaciones_rep = ds_ASP['UBICACIÓN']

# Diccionarios con IDs
distritos_dict = {}           # distrito -> id_distrito
ubicaciones_dict = {}         # (ubicacion, distrito_id) -> id_ubicacion

id_distrito = 1
id_ubicacion = 1

# =========================
# CATÁLOGOS CONTROLADOS
# =========================
CATALOGO_DISTRITOS = {
    "ATE",
    "SAN ISIDRO",
    "CAMACHO",
    "MOLINA"
}

CATALOGO_UBICACIONES = {
    "RETIRO DIRECTO",
    "SHOWROOM",
    "COCHERA",
    "FRONTIS",
    "PATIO",
    "SLA"
}

# FUNCIONES
def limpiar_texto(texto: str):
    texto = texto.upper()
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

def detectar_distrito(texto: str):
    for d in sorted(CATALOGO_DISTRITOS, key=len, reverse=True):
        if d in texto:
            return d
    return None

def detectar_ubicacion(texto: str):
    for u in sorted(CATALOGO_UBICACIONES, key=len, reverse=True):
        if u in texto:
            return u
    return None

for fila in ubicaciones_rep:

    if not isinstance(fila, str) or fila.strip() == "":
        continue

    fila = limpiar_texto(fila)

    distrito = detectar_distrito(fila)
    ubicacion = detectar_ubicacion(fila)

    if ubicacion and not distrito:
        distrito = random.choice(list(CATALOGO_DISTRITOS))

    # Registrar distrito
    if distrito and distrito not in distritos_dict:
        distritos_dict[distrito] = id_distrito
        id_distrito += 1

    # Registrar ubicación SOLO si tiene distrito
    if distrito and ubicacion:
        distrito_id = distritos_dict[distrito]

        key = (ubicacion, distrito_id)

        if key not in ubicaciones_dict:
            ubicaciones_dict[key] = id_ubicacion
            id_ubicacion += 1

sql_distritos = "-- migrate:up\n\n"

for nombre, id_val in distritos_dict.items():
    nombre_esc = nombre.replace("'", "''")
    sql_distritos += (
        f"INSERT INTO distritos (id_distrito, nombre) "
        f"VALUES ({id_val}, '{nombre_esc}');\n"
    )

sql_distritos += "\n-- migrate:down\nDELETE FROM distritos;\n"

with open("inserts_distritos.sql", "w", encoding="utf-8") as f:
    f.write(sql_distritos)


#UBICACIONES
sql_ubicaciones = "-- migrate:up\n\n"

for (ubicacion, distrito_id), id_val in ubicaciones_dict.items():
    ubic_esc = ubicacion.replace("'", "''")

    sql_ubicaciones += (
        "INSERT INTO ubicaciones "
        "(id_ubicacion, nombre, distritos_id) "
        f"VALUES ({id_val}, '{ubic_esc}', {distrito_id});\n"
    )

sql_ubicaciones += "\n-- migrate:down\nDELETE FROM ubicaciones;\n"

with open("inserts_ubicaciones.sql", "w", encoding="utf-8") as f:
    f.write(sql_ubicaciones)

<h2>Campañas</h2>

In [11]:
import re

campanas_rep = ds_ASP['CAMPAÑA']

campanas_diccionario = {}
id_campaña = 1

def campaña_valida(nombre):
    if not isinstance(nombre, str):
        return False

    nombre = nombre.strip().upper()

    if nombre == "" or nombre in {"SIN CAMPAÑA", "NO APLICA"}:
        return False

    return True

for c in campanas_rep:

    if not campaña_valida(c):
        continue

    nombre = c.strip().upper()

    # SOLO asignar ID si no existe aún
    if nombre not in campanas_diccionario:
        campanas_diccionario[nombre] = id_campaña
        id_campaña += 1

sql_cam = "-- migrate:up\n\n"

for nombre, id_val in sorted(campanas_diccionario.items(), key=lambda x: x[1]):

    descuento = 0.10
    nombre_sql = nombre.replace("'", "''")

    sql_cam += (
        "INSERT INTO campañas (id_campaña, nombre, descuento) "
        f"VALUES ({id_val}, '{nombre_sql}', {descuento});\n"
    )

sql_cam += "\n-- migrate:down\nDELETE FROM campañas;\n"

with open("inserts_campañas.sql", "w", encoding="utf-8") as f:
    f.write(sql_cam)

In [12]:
print(campanas_diccionario.items())

dict_items([('NO APLICA CAMPAÑA', 1), ('CAMPAÑA OK', 2), ('NO APLICA CAMPAÑA+REVISION TALLER OK', 3), ('HACER CAMPAÑA', 4), ('NO APLICA CAMPAÑA/REVISION TALLER OK', 5), ('CAMPAÑA+REVISION OK', 6), ('CAMPAÑA 25C42 SIN PROCEDIMIENTO', 7), ('HACER CAMPAÑA/SIN PROCEDIMIENTO', 8), ('CON CAMPAÑA SIN PROCEDIMIENTO', 9), ('HACER CAMPAÑA CORREO', 10), ('CAMPAÑA 8/1', 11), ('HACER CAMPAÑA 2/1', 12), ('CAMPAÑA 7/1', 13)])


<H2>Recepciones</h2>

<p>Antes de pasar a <e>recepciones_campañas</e> se debe crear recpeciones y campañas de forma individual, luego se vinculan entre ellos</p>

In [21]:
import pandas as pd
import numpy as np

receps = ds_ASP[
    ["PLACA",
     "FECHA DE INGRESO ",
     "FECHA TARJETA",
     "FECHA PLACA",
     "FECHA DE RECEPCION DEL VEHICULO",
     "UBICACIÓN",
     "Asesor",
     "CAMPAÑA"]
].to_numpy()

text = '-- migrate:up\n\n'

id_recepcion = 1
recepciones_diccionario = {}

for r in receps:

    placa = str(r[0]).strip().upper()
    vehiculo_id = 'NULL'

    for vid, datos in vehiculos_diccionario.items():
        if datos[0] == placa:
            vehiculo_id = vid

    # --- FECHAS
    def limpiar_fecha(valor):
        if pd.isna(valor):
            return 'NULL'
        fecha = pd.to_datetime(valor, errors='coerce')
        if pd.isna(fecha):
            return 'NULL'
        return f"'{fecha.strftime('%Y-%m-%d')}'"

    fecha_entrega   = limpiar_fecha(r[1])
    fecha_tarjeta   = limpiar_fecha(r[2])
    fecha_placa     = limpiar_fecha(r[3])
    
    fecha_recepcion = limpiar_fecha(r[4])
    # --- Ubicacion ---
    ubicacion_r = 'NULL'
    if pd.notna(r[5]):
        ubicacion_str = str(r[5]).strip().upper()
        for (nombre, distrito_id), mid in ubicaciones_dict.items():
            if nombre.upper() in ubicacion_str:
                ubicacion_r = mid

    # --- ASESOR
    asesor_str = str(r[6]).strip().upper()
    asesor_id = asesores_diccionario.get(asesor_str, 'NULL')

    # --- CAMPAÑA
    campaña_id = "NULL"
    if pd.notna(r[7]):
        campaña_str = str(r[7]).strip().upper()
        campaña_id = campanas_diccionario.get(campaña_str, 'NULL')

    valores = [
        fecha_entrega,
        fecha_tarjeta,
        fecha_placa,
        fecha_recepcion,
        ubicacion_r,
        asesor_id,
        vehiculo_id,
        campaña_id
    ]

    if all(v == 'NULL' for v in valores):
        continue

    # --- INSERT SQL
    text += (
        f"INSERT INTO recepciones "
        f"(id_recepcion, fecha_entrega, fecha_tarjeta, fecha_placa, fecha_recepcion, ubicacion_id, asesor_id, vehiculo_id, campaña_id) "
        f"VALUES ({id_recepcion}, {fecha_entrega}, {fecha_tarjeta}, {fecha_placa}, {fecha_recepcion}, "
        f"{ubicacion_r}, {asesor_id}, {vehiculo_id}, {campaña_id});\n"
    )

    # --- GUARDAR DICCIONARIO
    recepciones_diccionario[id_recepcion] = (
        fecha_entrega,
        fecha_tarjeta,
        fecha_placa,
        fecha_recepcion,
        ubicacion_r,
        asesor_id,
        vehiculo_id,
        campaña_id
    )

    id_recepcion += 1

text += '\n-- migrate:down\nDELETE FROM recepciones;\n'

with open('inserts_recepciones.sql', 'w', encoding='utf-8') as archivo:
    archivo.write(text)


C:\Users\DELL\AppData\Local\Temp\ipykernel_300\3392632149.py:33: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fecha = pd.to_datetime(valor, errors='coerce')


filtrado telefono

In [14]:
import re
import pandas as pd

def extraer_telefonos(texto):
    if texto is None:
        return None

    numeros = re.sub(r'\D', '', str(texto))

    if numeros.startswith("51"):
        numeros = numeros[2:]

    bloques = [numeros[i:i+9] for i in range(0, len(numeros), 9)]
    bloques = [b for b in bloques if len(b) == 9]

    return "|".join(bloques) if bloques else None


telefonos_rep = ds_ASP['Celular'].to_numpy()

telefonos_dict = {} 
id_telefono = 1
sql_tel = '-- migrate:up\n\n'

for n in telefonos_rep:

    if pd.isna(n):
        continue

    lista_telefonos = extraer_telefonos(n)

    if lista_telefonos is None:
        continue

    sql_tel += (
        f"INSERT INTO telefonos "
        f"(id_telefono, numero, cliente_id) "
        f"VALUES ({id_telefono}, '{lista_telefonos}');\n"
    )

    telefonos_dict[id_telefono] = lista_telefonos
    id_telefono += 1


sql_tel += "\n-- migrate:down\nDELETE FROM telefonos;\n"

with open("inserts_telefono.sql", "w", encoding="utf-8") as f:
    f.write(sql_tel)

<h3>Telefono - Cliente asociacion </h3>

In [ ]:
import re
import pandas as pd


telefonos_clientes = ds_ASP[['Celular', 'Cliente']].to_numpy()

telefonosCliente_dict = {} 
id_telefono_cliente = 1
sql_tel = '-- migrate:up\n\n'

for n in telefonos_clientes:
    if pd.isna(n[1]):
        continue

    nombre_cliente = str(n[1]).strip().upper()
    cliente_id_T = None
    for cid, (_, nombre) in clientes_dict.items():
        if nombre.upper() in nombre_cliente:
            cliente_id_T = cid
            break

    if cliente_id_T is None:
        continue

    if cliente_id_T in telefonosCliente_dict:
        continue

    lista_telefonos = extraer_telefonos(n[0])

    if lista_telefonos is None:
        continue

    sql_tel += (
        f"INSERT INTO cliente_telefono "
        f"(id_telefono_cliente, cliente_id) "
        f"VALUES ({id_telefono_cliente}, {cliente_id_T});\n"
    )

    telefonosCliente_dict[cliente_id_T] = id_telefono_cliente
    id_telefono_cliente += 1


sql_tel += "\n-- migrate:down\nDELETE FROM telefonos;\n"

with open("insert_cliente_telefono.sql", "w", encoding="utf-8") as f:
    f.write(sql_tel)

filtrado formato pago

In [16]:
import pandas as pd

formas_unicas = ds_ASP['FORMA DE PAGO'].dropna().unique()

formas_unicas = sorted(formas_unicas)  



formatos_diccionario = {}



text = '-- migrate:up \n\n'

id_counter = 1

for forma in formas_unicas:

    forma_escaped = str(forma).replace("'", "''").strip()

    if forma_escaped:  # evita vacíos

        text += f"INSERT INTO formaPago (id_forma_pago, nombre) VALUES ({id_counter}, '{forma_escaped}');\n"

        formatos_diccionario[forma_escaped] = id_counter

        id_counter += 1



text += '\n-- migrate:down \n\nDELETE FROM formatos_pago;'



with open('inserts_formatos_pago.sql', 'w', encoding='utf-8') as f:

    f.write(text)



print("Diccionario generado (para usarlo después en pagos):")

print(formatos_diccionario)

print(f"\nTotal de formatos únicos: {len(formatos_diccionario)}")

Diccionario generado (para usarlo después en pagos):
{'AUTOPLAN': 1, 'AUTOPLAN VB ENTREGA+BCP': 2, 'BBVA': 3, 'BBVA BCP': 4, 'BBVA BCP INTBK': 5, 'BBVA BCP INTBK SCTBK': 6, 'BBVA BCP INTERBANK': 7, 'BBVA BCP ITBK': 8, 'BBVA BCP ITBK NIUBIZ': 9, 'BBVA BCP SCTBK': 10, 'BBVA IBK BCP': 11, 'BBVA INTBK': 12, 'BBVA INTBK BCP': 13, 'BBVA INTERBANK': 14, 'BBVA ITBK': 15, 'BBVA NIUBIZ': 16, 'BBVA SCOTBK': 17, 'BBVA SCOTIABANK': 18, 'BBVA SCOTIABANK MAF': 19, 'BBVA SCTBK BCP': 20, 'BBVA VISA': 21, 'BCP': 22, 'BCP BBVA': 23, 'BCP BBVA INTBK': 24, 'BCP BBVA MAF': 25, 'BCP INTBK': 26, 'BCP INTBK FONBIENES': 27, 'BCP INTBK NIUBIZ': 28, 'BCP INTERBANK': 29, 'BCP INTERBANK SCOTIABANK BBVA': 30, 'BCP ITBK': 31, 'BCP ITBK NIUBIZ': 32, 'BCP NIUBIZ': 33, 'BCP NIUBIZ+AUTOPLAN PEDIR VB': 34, 'BCP SANTANDER': 35, 'BCP SCOTBK': 36, 'BCP SCOTIABANK': 37, 'BCP SCTBK': 38, 'BCP SCTBK INTBK': 39, 'BCP VISA': 40, 'BCP VISA+PANDERO': 41, 'BCP+AUTOPLAN': 42, 'BCP+AUTOPLAN PEDIR VB PARA ENTREGA': 43, 'BCP+MAQUIMAS': 

Filtrado bancos

In [17]:
import pandas as pd
import re

bancos_conocidos = [
    'BCP', 'BBVA', 'SANTANDER', 'INTERBANK', 'INTBK', 'ITBK', 'IBK',
    'SCOTIABANK', 'SCTBK', 'SCOTBK', 'NIUBIZ',
    'PANDERO', 'VISA', 'BANBIF'
]



normalization = {
    'IBK': 'INTERBANK',
    'INTBK': 'INTERBANK',
    'ITBK': 'INTERBANK',
    'SCOTBK': 'SCOTIABANK',
    'SCTBK': 'SCOTIABANK',
    'NIUBIS': 'NIUBIZ',
}

def extraer_bancos(texto):
    if pd.isna(texto):
        return set()
    texto_upper = str(texto).upper()
    encontrados = set()

    for banco in bancos_conocidos:
        if re.search(r'\b' + re.escape(banco) + r'\b', texto_upper) or banco in texto_upper:
            encontrados.add(banco)

    if 'LEASING' in texto_upper:
        encontrados.add('SANTANDER')

    normalized = {normalization.get(b, b) for b in encontrados}
    return normalized


all_bancos = set()
for forma in ds_ASP['FORMA DE PAGO'].dropna():
    all_bancos.update(extraer_bancos(forma))
bancos_unicos = sorted(list(all_bancos))

bancos_diccionario = {}
text = '-- migrate:up \n\n'
id_counter = 1
for banco in bancos_unicos:
    banco_escaped = banco.replace("'", "''")
    text += f"INSERT INTO bancos (id_banco, nombre) VALUES ({id_counter}, '{banco_escaped}');\n"
    bancos_diccionario[banco] = id_counter
    id_counter += 1
text += '\n-- migrate:down \n\nDELETE FROM bancos;'
with open('inserts_bancos.sql', 'w', encoding='utf-8') as f:
    f.write(text)

print("Bancos únicos encontrados:", bancos_unicos)
print("Diccionario de bancos:", bancos_diccionario)
print(f"Total de bancos únicos: {len(bancos_unicos)}")

Bancos únicos encontrados: ['BANBIF', 'BBVA', 'BCP', 'INTERBANK', 'NIUBIZ', 'PANDERO', 'SANTANDER', 'SCOTIABANK', 'VISA']
Diccionario de bancos: {'BANBIF': 1, 'BBVA': 2, 'BCP': 3, 'INTERBANK': 4, 'NIUBIZ': 5, 'PANDERO': 6, 'SANTANDER': 7, 'SCOTIABANK': 8, 'VISA': 9}
Total de bancos únicos: 9


Filtro pagos:

In [18]:
import pandas as pd
import random

print("Verificación diccionarios:")
print(f"  Recepciones: {len(recepciones_diccionario) if 'recepciones_diccionario' in globals() else 'NO EXISTE'}")
print(f"  Formatos:    {len(formatos_diccionario) if 'formatos_diccionario' in globals() else 'NO EXISTE'}")
print(f"  Bancos:      {len(bancos_diccionario) if 'bancos_diccionario' in globals() else 'NO EXISTE'}")

if not all([
    'recepciones_diccionario' in globals() and len(recepciones_diccionario) > 0,
    'formatos_diccionario' in globals() and len(formatos_diccionario) > 0,
    'bancos_diccionario' in globals() and len(bancos_diccionario) > 0
]):
    print("\n¡Faltan diccionarios! Ejecuta primero los códigos de recepciones, formatos y bancos.")
else:
    print("\nTodos los diccionarios están listos → procediendo a generar pagos...")

def obtener_banco_principal(forma_str):
    if not isinstance(forma_str, str):
        return 'BCP'
    texto = forma_str.upper()
    orden_prioridad = ['INTERBANK', 'SCOTIABANK', 'BBVA', 'BCP', 'SANTANDER', 'NIUBIZ']
    for banco in orden_prioridad:
        if banco in texto:
            return banco
    for banco in bancos_diccionario:
        if banco in texto:
            return banco
    return 'BCP'


text = '-- migrate:up \n\n'
id_pago = 1
errores = 0

for idx, row in ds_ASP.iterrows():
    if not recepciones_diccionario:
        errores += 1
        continue

    recepcion_id = (id_pago % len(recepciones_diccionario)) + 1  
    forma_raw = row.get('FORMA DE PAGO')
    forma_str = str(forma_raw).strip() if pd.notna(forma_raw) else ''
    forma_escaped = forma_str.replace("'", "''")

    formas_pago_id = formatos_diccionario.get(forma_escaped)
    if formas_pago_id is None:
        errores += 1
        continue

    
    banco_nombre = obtener_banco_principal(forma_str)
    bancos_id = bancos_diccionario.get(banco_nombre)
    if bancos_id is None:
        errores += 1
        continue

    monto = round(random.uniform(8000, 85000), 2)

    text += (
        f"INSERT INTO pagos (id_pago, recepcion_id, monto, bancos_id, formas_pago_id) "
        f"VALUES ({id_pago}, {recepcion_id}, {monto}, {bancos_id}, {formas_pago_id});\n"
    )

    id_pago += 1

text += '\n-- migrate:down \n\nDELETE FROM pagos;\n'

with open('inserts_pagos.sql', 'w', encoding='utf-8') as f:
    f.write(text)

print(f"\nResultado final:")
print(f"Pagos generados: {id_pago - 1}")
print(f"Errores / filas saltadas: {errores}")
print("Archivo creado: inserts_pagos.sql")

Verificación diccionarios:
  Recepciones: 1258
  Formatos:    195
  Bancos:      9

Todos los diccionarios están listos → procediendo a generar pagos...

Resultado final:
Pagos generados: 1004
Errores / filas saltadas: 254
Archivo creado: inserts_pagos.sql
